<a href="https://colab.research.google.com/github/welsongds/desafio_dio_analisedados/blob/main/Desafio_DIO_analisedados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = "https://sdw-2023-prd.up.railway.app"

## Extract
Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.


In [16]:
import pandas as pd

df = pd.read_csv('/content/SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)


[2110, 2111, 2113]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

## Transform
Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuáio.




In [ ]:
!pip install openai

In [23]:
openai_api_key = 'KEY_API_OPENAI'

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"}
    ]
  )
  return completion.choice[0].message.content.strip('/"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
      "description": news
  })

## Load
Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/user/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  sucess = update_user(user)
  print(f"User {user['name']} updated? {sucess}!")